In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
# one_hot 옵션은 레이블을 동물 분류 예제에서 보았던 one_hot 방식의 데이터로 만들어줍니다.
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)


In [ ]:
# 신경망 모델 구성
# data-> model -> cost / optimizer
# 28 x 28

global_step = tf.Variable(0, trainable=False, name='global_step')
X=tf.placeholder(tf.float32,[None,784])
# [1 0 0 0 0 0 0 0]
Y=tf.placeholder(tf.float32,[None,10])

# 신경망의 레이어는 다음처럼 구성
# 784 -> 256 -> 256 -> 10
W1=tf.Variable(tf.random_normal([784,256],stddev=0.01))
L1=tf.nn.relu(tf.matmul(X,W1))

W2=tf.Variable(tf.random_normal([256,256],stddev=0.01))
L2=tf.nn.relu(tf.matmul(L1,W2))

W3=tf.Variable(tf.random_normal([256,10],stddev=0.01))
model=tf.matmul(L2,W3)

cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model,labels=Y))
optimizer=tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

# 신경망 모델 학습
batch_size=100
total_batch=int(mnist.train.num_examples/batch_size)

with tf.Session() as sess:
    
    # 모델 저장
    saver=tf.train.Saver(tf.global_variables())
    ckpt=tf.train.get_checkpoint_state('./model')
    if ckpt and  tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
        print("저장된 checkpoint가 있습니다.")
        sess.run(tf.global_variables_initializer())
        # 마지막 파일 명에서 저장된 STEP 받아오기
        global_step = int(ckpt.model_checkpoint_path.split('-')[1].split('.')[0])
        saver.restore(sess,ckpt.model_checkpoint_path)
        #saver.restore(sess, tf.train.latest_checkpoint('./') )
        
    else:
        print("저장된 checkpoint가 없습니다.")
        sess.run(tf.global_variables_initializer())
        global_step=0
        
    for epoch in range(global_step,20):
        total_cost=0
        
        for i in range(total_batch):
            batch_xs,batch_ys=mnist.train.next_batch(batch_size)
            
            _,cost_val=sess.run([optimizer,cost],feed_dict={X:batch_xs, Y:batch_ys})
            total_cost+=cost_val
            
        print('Epoch:' , '%04d' %(epoch+1), 'Avg.cost =','{:.3f}',format(total_cost/total_batch))
        print("모델 저장")
        if (epoch+1)%5==0:
            saver.save(sess,"./model/mnist_model",global_step=epoch)
        
    print("최적화 완료")
    # 결과 확인
    is_correct=tf.equal(tf.argmax(model,1), tf.argmax(Y,1))
    accuracy=tf.reduce_mean(tf.cast(is_correct, tf.float32))
    print('정확도 :',sess.run(accuracy, feed_dict={X:mnist.test.images, Y:mnist.test.labels}))
            
        
        

